<a href="https://colab.research.google.com/github/roberto-adrego/Pos_dsa/blob/main/Projeto2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
## <font color='blue'>IA Generativa e LLMs Para Processamento de Linguagem Natural</font>
## <font color='blue'>Projeto 2</font>
## <font color='blue'>Fine-Tuning Eficiente de LLMs com LoRA Para Análise de Sentimentos em Texto</font>

## Instalando e Carregando Pacotes

In [ ]:
!pip install -q -U watermark

A descrição de cada um dos pacotes abaixo está disponível no Capítulo 7 do Curso.

https://www.datascienceacademy.com.br/course/ia-generativa-e-llms-para-processamento-de-linguagem-natural

In [ ]:
!pip install -q accelerate==1.9.0 peft==0.16.0 bitsandbytes==0.46.1 transformers==4.54.0

In [ ]:
!pip install -q trl==0.20.0 gradio==5.38.2 protobuf scipy==1.16.0

In [ ]:
!pip install -q sentencepiece==0.2.0 tokenizers==0.21.2 datasets==4.0.0

In [ ]:
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



In [ ]:
# Imports
import os
import torch
import datasets
import pandas as pd
from datasets import load_dataset
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          HfArgumentParser,
                          TrainingArguments,
                          pipeline,
                          logging)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, SFTConfig
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Define o nível de log para CRITICAL
logging.set_verbosity(logging.CRITICAL)

In [ ]:
# Verifica o modelo da GPU
if torch.cuda.is_available():
    print('Número de GPUs:', torch.cuda.device_count())
    print('Modelo GPU:', torch.cuda.get_device_name(0))
    print('Total Memória [GB] da GPU:',torch.cuda.get_device_properties(0).total_memory / 1e9)

Número de GPUs: 1
Modelo GPU: NVIDIA A100-SXM4-40GB
Total Memória [GB] da GPU: 42.474471424


In [ ]:
# Reset da memória da GPU
from numba import cuda
device = cuda.get_current_device()
device.reset()

In [ ]:
# Define o nome do dataset
nome_dataset = "dataset.csv"

In [ ]:
# Carrega os dados
dataset_carregado = load_dataset('csv', data_files = nome_dataset, delimiter = ',')

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# Dados Carregados no Formato de Dicionário
dataset_carregado

DatasetDict({
    train: Dataset({
        features: ['train'],
        num_rows: 17057
    })
})

https://huggingface.co/NousResearch/Llama-2-7b-chat-hf

In [ ]:
# Nome do repositório do LLM pré-treinado
repositorio_hf = "NousResearch/Llama-2-7b-chat-hf"

In [ ]:
# Nome do novo modelo
modelo_dsa = "novo_modelo_dsa"

## Definindo os Parâmetros de Configuração

In [ ]:
# Parâmetros LoRA
lora_r = 32
lora_alpha = 16
lora_dropout = 0.1

Os parâmetros acima são do LoRa (Low-Rank Adaptation), a parte base do QLoRA (Quantized Low-Rank Adaptation), uma técnica utilizada para adaptar modelos de linguagem de forma eficiente. Acima dos parâmetros LoRa colocamos os parâmetros de quantização, definindo assim o QLoRa.

Vamos descrever cada um dos parâmetros:

**lora_r**: Este parâmetro representa o "rank" na adaptação de Low-Rank (LoRA). Um valor de 32 significa que a matriz de pesos do modelo original será aproximada por duas matrizes menores cujo produto possui um rank máximo de 32. Essencialmente, isso reduz a complexidade computacional e o número de parâmetros a serem treinados durante a adaptação, mantendo a eficácia do modelo.

**lora_alpha**: Este é um fator de escala que é aplicado às atualizações de peso do LoRA durante o treinamento. Um valor de 16 indica que as atualizações de pesos serão escaladas por este fator. Esse parâmetro é importante porque permite um controle fino sobre a magnitude das atualizações dos pesos, o que pode afetar a rapidez e a eficácia da adaptação do modelo.

**lora_dropout**: Este parâmetro representa a taxa de "dropout" aplicada durante a adaptação do modelo. O valor 0.1 significa que 10% das unidades serão aleatoriamente descartadas (ou "desligadas") durante o treinamento. O dropout é uma técnica comum para evitar o overfitting em redes neurais, garantindo que o modelo não se torne excessivamente dependente de qualquer parte específica dos dados de treinamento.

In [ ]:
# Parâmetros bitsandbytes (QLoRa)
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

Os parâmetros acima são para a biblioteca bitsandbytes, uma ferramenta para otimização de treinamento de modelos de aprendizado de máquina, em particular para reduzir o uso de memória e acelerar o treinamento. Aqui está a explicação de cada parâmetro:

**use_4bit**: Este parâmetro indica se a quantização de 4 bits deve ser usada ou não. Ao definir True, isso significa que o modelo irá utilizar uma representação de 4 bits para os pesos durante o treinamento. Isso reduz significativamente a quantidade de memória necessária, permitindo treinar modelos maiores ou reduzir os requisitos de hardware.

**bnb_4bit_compute_dtype**: Este é o tipo de dado usado para cálculos durante o treinamento quando a quantização de 4 bits está ativa. O valor float16 significa que os cálculos serão feitos usando números de ponto flutuante de 16 bits. Isso é geralmente usado para equilibrar a eficiência computacional e a precisão numérica.

**bnb_4bit_quant_type**: Especifica o tipo de quantização a ser usado. O valor nf4 é um tipo específico de quantização desenvolvido pela bitsandbytes, otimizado para eficiência e eficácia em treinamento de modelos de aprendizado profundo. Este tipo de quantização é projetado para manter a precisão do modelo enquanto reduz os requisitos de memória.

**use_nested_quant**: Indica se uma técnica de quantização aninhada será usada. False significa que essa técnica não será empregada. A quantização aninhada pode ser usada para reduzir ainda mais o uso de memória, aplicando diferentes níveis de quantização a diferentes partes do modelo, mas pode ser mais complexa de implementar e gerenciar.

Esses parâmetros são usados para configurar como o modelo de aprendizado profundo irá lidar com a representação e cálculo dos pesos durante o treinamento, visando otimizar o uso de memória e acelerar o processo de treinamento.

In [ ]:
# Parâmetros do ajuste fino
output_dir = "saida"
num_train_epochs = 1
fp16 = True
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"
max_steps = -1
warmup_ratio = 0.03

Os parâmetros acima são usados para configurar o processo de ajuste fino (fine-tuning) de modelos de aprendizado de máquina, especialmente modelos de linguagem natural. Vamos descrever cada um deles:

**output_dir**: Especifica o diretório onde os resultados do treinamento serão salvos.

**num_train_epochs**: O número de épocas de treinamento. O valor 1 significa que o modelo passará uma vez por todo o conjunto de dados de treinamento.

**fp16**: Indica se deve ser usado o treinamento com precisão mista de ponto flutuante de 16 bits (FP16). O valor True significa que sim, o que pode acelerar o treinamento e reduzir o uso de memória, mantendo uma precisão aceitável.

**bf16**: Semelhante ao fp16, mas para o formato bfloat16. False significa que não será utilizado. O formato bfloat16 é outra forma de reduzir o uso de memória e acelerar o treinamento, com impactos ligeiramente diferentes na precisão. Podemos usar fp16 ou bf16, mas não podemos usar ambos simultaneamente.

**per_device_train_batch_size**: Tamanho do lote de treinamento por dispositivo. O valor 4 indica que cada dispositivo de treinamento (como uma GPU) processará 4 exemplos por lote.

**per_device_eval_batch_size**: Tamanho do lote de avaliação por dispositivo, também definido como 4.

**gradient_accumulation_steps**: Número de passos para acumulação de gradientes antes de realizar uma atualização de parâmetros. O valor 1 significa que não há acumulação (cada passo resulta em uma atualização).

**gradient_checkpointing**: Habilita o checkpointing de gradientes, que é uma técnica para reduzir o uso de memória ao custo de um tempo de treinamento ligeiramente maior. True indica que está habilitado.

**max_grad_norm**: Norma máxima para o corte de gradientes. O valor 0.3 é um valor que ajuda a evitar o problema de explosão de gradientes em treinamentos.

**learning_rate**: Taxa de aprendizado inicial. O valor 2e-4 é um valor comum para ajuste fino, proporcionando um equilíbrio entre a velocidade de aprendizado e a estabilidade.

**weight_decay**: Taxa de decaimento de peso, usada para regularização. O valor 0.001 é um valor que ajuda a prevenir o overfitting.

**optim**: O otimizador usado. "paged_adamw_32bit" é uma variante do AdamW otimizado para eficiência em termos de memória.

**lr_scheduler_type**: Tipo de agendador de taxa de aprendizado. O valor "cosine" indica o uso do agendador cosseno, que ajusta a taxa de aprendizado seguindo uma curva cosseno.

**max_steps**: Número máximo de passos de treinamento. O valor -1 significa que o treinamento continuará até que o número de épocas seja alcançado.

**warmup_ratio**: Proporção do número total de passos de treinamento usados para o aquecimento linear da taxa de aprendizado. O valor 0.03 significa que 3% do treinamento inicial será usado para aumentar gradualmente a taxa de aprendizado.

Esses parâmetros são essenciais para configurar de maneira eficiente o processo de ajuste fino, impactando diretamente na qualidade do modelo treinado, no tempo de treinamento e no uso de recursos computacionais.

In [ ]:
# Agrupando sequências em lotes de mesmo comprimento
group_by_length = True
save_steps = 0
logging_steps = 400

<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
Os parâmetros acima são usados para configurar certos aspectos do processo de treinamento de modelos de aprendizado de máquina, especialmente modelos de linguagem. Eles estão relacionados a como as sequências são agrupadas em lotes e como o progresso do treinamento é registrado e salvo. Vamos detalhar cada um:

**group_by_length**: Este parâmetro indica se as sequências devem ser agrupadas por comprimento ao formar lotes de treinamento. Quando True, isso significa que o treinamento agrupará sequências de comprimentos semelhantes juntas em um lote. Esta é uma prática eficiente porque reduz a quantidade de preenchimento (padding) necessário. O preenchimento é usado para garantir que todas as sequências em um lote tenham o mesmo comprimento, mas pode ser um desperdício de recursos computacionais. Agrupar sequências de comprimentos semelhantes minimiza esse desperdício.

**save_steps**: Especifica a frequência com que o modelo treinado deve ser salvo. Um valor de 0 indica que o modelo não será salvo automaticamente com base em um número de passos. Em vez disso, o modelo pode ser salvo no final de cada época de treinamento ou manualmente.

**logging_steps**: Define a frequência com que as informações de log devem ser registradas. O valor 400 significa que o processo de treinamento registrará informações como a perda de treinamento (loss), métricas de avaliação, entre outros, a cada 400 passos de treinamento. Isso é útil para monitorar o progresso do treinamento e para o ajuste fino dos hiperparâmetros.

In [ ]:
# Precisão dos dados para treinamento
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

O código acima está relacionado à configuração do tipo de dado (dtype) para computação durante o treinamento de modelos de rede neural usando a biblioteca PyTorch. Vamos analisar cada parte do código:

**torch**: É uma referência à biblioteca PyTorch, uma biblioteca para aprendizado de máquina e redes neurais.

**bnb_4bit_compute_dtype**: Esta é uma variável que armazena uma string representando o tipo de dado desejado para computação. O parâmetro bnb_4bit_compute_dtype foi definido como "float16", indicando que a computação deve ser realizada usando números de ponto flutuante de 16 bits.

**getattr**: É uma função Python built-in usada para obter um atributo de um objeto. Neste caso, ela está sendo usada para obter um atributo da biblioteca PyTorch com base no valor da string armazenada em bnb_4bit_compute_dtype.

O que acontece aqui é que getattr(torch, bnb_4bit_compute_dtype) recupera o tipo de dados de ponto flutuante de 16 bits (torch.float16) da biblioteca PyTorch, com base no valor de bnb_4bit_compute_dtype. Em seguida, esse tipo de dados é atribuído à variável compute_dtype.

O uso de compute_dtype no treinamento de modelos de rede neural tem implicações importantes:

**Eficiência de Memória**: Usar float16 ao invés de tipos de dados mais comuns como float32 pode reduzir significativamente o uso de memória, permitindo o treinamento de modelos maiores ou a execução de mais processos em paralelo.

**Velocidade de Computação**: Muitas GPUs modernas têm otimizações para cálculos float16, o que pode acelerar o treinamento.

**Precisão**: Embora float16 possa ser menos preciso do que float32, muitas vezes é suficientemente preciso para tarefas de treinamento de modelos de rede neural.

In [ ]:
# Definindo os parâmetros da quantização
bnb_config = BitsAndBytesConfig(load_in_4bit = use_4bit,
                                bnb_4bit_quant_type = bnb_4bit_quant_type,
                                bnb_4bit_compute_dtype = compute_dtype,
                                bnb_4bit_use_double_quant = use_nested_quant)

In [ ]:
# Carregando o modelo base pré-treinado
modelo = AutoModelForCausalLM.from_pretrained(repositorio_hf,
                                              quantization_config = bnb_config,
                                              device_map = "auto")

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [ ]:
# Não usaremos o cache
modelo.config.use_cache = False
modelo.config.pretraining_tp = 1

In [ ]:
# Carregando o tokenizador do modelo base
tokenizador = AutoTokenizer.from_pretrained(repositorio_hf, trust_remote_code = True)
tokenizador.pad_token = tokenizador.eos_token
tokenizador.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
# Carregando a configuração LoRA
peft_config = LoraConfig(lora_alpha = lora_alpha,
                         lora_dropout = lora_dropout,
                         r = lora_r,
                         bias = "none",
                         task_type = "CAUSAL_LM")

https://github.com/huggingface/peft/blob/main/src/peft/tuners/lora/config.py

In [ ]:
# Definindo os parâmetros de treino
training_arguments = TrainingArguments(output_dir = output_dir,
                                       num_train_epochs = num_train_epochs,
                                       per_device_train_batch_size = per_device_train_batch_size,
                                       gradient_accumulation_steps = gradient_accumulation_steps,
                                       optim = optim,
                                       save_steps = save_steps,
                                       logging_steps = logging_steps,
                                       learning_rate = learning_rate,
                                       weight_decay = weight_decay,
                                       fp16 = fp16,
                                       bf16 = bf16,
                                       max_grad_norm = max_grad_norm,
                                       max_steps = max_steps,
                                       warmup_ratio = warmup_ratio,
                                       group_by_length = group_by_length,
                                       lr_scheduler_type = lr_scheduler_type)

Veja a descrição de Supervised Fine-Tuning e RLHF no videobook.

In [ ]:
# Definindo os Parâmetros do Fine-Tuning Supervisionado
sft_config = SFTConfig(per_device_train_batch_size = 2,
                       gradient_accumulation_steps = 4,
                       warmup_steps = 5,
                       num_train_epochs = 1,
                       learning_rate = 2e-4,
                       fp16 = True,
                       logging_steps = 1,
                       optim = "adamw_8bit",
                       weight_decay = 0.01,
                       lr_scheduler_type = "linear",
                       seed = 3407,
                       output_dir = "outputs",
                       report_to = "none",
                       dataset_text_field = "train",
                       packing = False)

In [ ]:
# Definindo os Parâmetros do Fine-Tuning Supervisionado (requer os parâmetros gerais no sft_config acima)
dsa_trainer = SFTTrainer(model = modelo,
                         train_dataset = dataset_carregado['train'],
                         peft_config = peft_config,
                         args = sft_config)

Adding EOS to train dataset:   0%|          | 0/17057 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/17057 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/17057 [00:00<?, ? examples/s]

https://huggingface.co/docs/trl/en/sft_trainer

> Treinamento do Modelo com o Ajuste Fino

In [ ]:
%%time
dsa_trainer.train()

{'loss': 0.9522, 'grad_norm': nan, 'learning_rate': 0.0, 'num_tokens': 786.0, 'mean_token_accuracy': 0.3002595081925392, 'epoch': 0.00046898815804900925}
{'loss': 0.8503, 'grad_norm': nan, 'learning_rate': 0.0, 'num_tokens': 1752.0, 'mean_token_accuracy': 0.280731949955225, 'epoch': 0.0009379763160980185}
{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 0.0, 'num_tokens': 2904.0, 'mean_token_accuracy': 0.2902160510420799, 'epoch': 0.0014069644741470278}
{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 0.0, 'num_tokens': 3516.0, 'mean_token_accuracy': 0.24193474650382996, 'epoch': 0.001875952632196037}
{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 0.0, 'num_tokens': 4431.0, 'mean_token_accuracy': 0.2713005356490612, 'epoch': 0.0023449407902450463}
{'loss': 0.4867, 'grad_norm': nan, 'learning_rate': 0.0, 'num_tokens': 5104.0, 'mean_token_accuracy': 0.19513889029622078, 'epoch': 0.0028139289482940555}
{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 0.0, 'num_tokens': 5980.0, 'mean_tok

TrainOutput(global_step=2133, training_loss=1.1741799978906986, metrics={'train_runtime': 2400.7478, 'train_samples_per_second': 7.105, 'train_steps_per_second': 0.888, 'total_flos': 1.1664325609849651e+17, 'train_loss': 1.1741799978906986})

In [ ]:
# Salvando o modelo treinado
dsa_trainer.model.save_pretrained(modelo_dsa)

<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->

In [ ]:
# Novo texto de entrada
prompt = "It's rare that a movie lives up to its hype, even rarer that the hype is transcended by the actual achievement"

https://huggingface.co/docs/transformers/en/main_classes/pipelines

In [ ]:
# Pipeline de Análise de Sentimentos com o Modelo Ajustado
pipe = pipeline(task = "text-generation",
                model = modelo,
                tokenizer = tokenizador,
                max_length = 200)

In [ ]:
# Executa o pipeline e extrai o resultado
resultado = pipe(f"<s>[INST] {prompt} [/INST]")

In [ ]:
print(resultado)

[{'generated_text': "<s>[INST] It's rare that a movie lives up to its hype, even rarer that the hype is transcended by the actual achievement [/INST]  I couldn't agree more! It's often the case that movies fail to meet the high expectations set by their marketing campaigns, but when a film manages to exceed those expectations, it's truly special. nobody likes a disappointing movie, but when a film exceeds expectations, it can be a truly magical experience.\n\nThere are several reasons why a movie might live up to or even surpass its hype. Here are a few:\n\n1. Originality: When a movie offers something new and unique, it can generate a lot of buzz and excitement. If the movie delivers on its promise of originality, it can exceed expectations and leave audiences feeling impressed and satisfied.\n2."}]


In [ ]:
print(resultado[0]['generated_text'])

<s>[INST] It's rare that a movie lives up to its hype, even rarer that the hype is transcended by the actual achievement [/INST]  I couldn't agree more! It's often the case that movies fail to meet the high expectations set by their marketing campaigns, but when a film manages to exceed those expectations, it's truly special. nobody likes a disappointing movie, but when a film exceeds expectations, it can be a truly magical experience.

There are several reasons why a movie might live up to or even surpass its hype. Here are a few:

1. Originality: When a movie offers something new and unique, it can generate a lot of buzz and excitement. If the movie delivers on its promise of originality, it can exceed expectations and leave audiences feeling impressed and satisfied.
2.


In [ ]:
# Libera a memória
del modelo
del pipe
del dsa_trainer
import gc
gc.collect()

60

In [ ]:
# Carrega o modelo em fp16 e faz o merge com os pesos LoRA
base_model = AutoModelForCausalLM.from_pretrained(repositorio_hf,
                                                  low_cpu_mem_usage = True,
                                                  return_dict = True,
                                                  torch_dtype = torch.float16,
                                                  device_map = "auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Cria o modelo final
modelo_dsa_final = PeftModel.from_pretrained(base_model, modelo_dsa)

In [ ]:
# Faz o merge e descarrega o modelo
modelo_dsa_final = modelo_dsa_final.merge_and_unload()

In [ ]:
# Carrega o tokenizador
tokenizador_dsa = AutoTokenizer.from_pretrained(repositorio_hf, trust_remote_code = True)
tokenizador_dsa.pad_token = tokenizador_dsa.eos_token
tokenizador_dsa.padding_side = "right"

In [ ]:
# Salva modelo e tokenizador
modelo_dsa_final.save_pretrained('novo_modelo-dsa-llm-projeto2')
tokenizador_dsa.save_pretrained('novo_modelo-dsa-llm-projeto2')

('novo_modelo-dsa-llm-projeto2/tokenizer_config.json',
 'novo_modelo-dsa-llm-projeto2/special_tokens_map.json',
 'novo_modelo-dsa-llm-projeto2/tokenizer.model',
 'novo_modelo-dsa-llm-projeto2/added_tokens.json',
 'novo_modelo-dsa-llm-projeto2/tokenizer.json')

In [ ]:
# Novo texto de entrada
prompt = "It's rare that a movie lives up to its hype, even rarer that the hype is transcended by the actual achievement"

In [ ]:
# Cria o pipeline
pipe = pipeline(task = "text-generation",
                model = modelo_dsa_final,
                tokenizer = tokenizador_dsa,
                max_length = 200)

In [ ]:
# Executa o pipeline e extrai o resultado
resultado = pipe(f"<s>[INST] {prompt} [/INST]")

In [ ]:
print(resultado)

[{'generated_text': "<s>[INST] It's rare that a movie lives up to its hype, even rarer that the hype is transcended by the actual achievement [/INST]  You're absolutely right! It's not uncommon for movies to generate a lot of buzz and excitement before their release, but often the final product doesn't quite live up to the hype. Here are some reasons why this happens:\n\n1. Overhyping: Sometimes, the media and fans can get carried away with the excitement surrounding a movie, building it up to be something it's not. This can create unrealistic expectations that are difficult for the movie to meet.\n2. High expectations: When a movie is highly anticipated, the pressure to deliver can be intense. This can lead to a situation where the movie is judged more harshly than it would be if it were a less hyped film."}]


In [ ]:
# Vamos não apenas classificar o sentimento.
# Vamos gerar texto positivo e/ou negativo a partir da avaliação (texto) inicial.
print(resultado[0]['generated_text'])

<s>[INST] It's rare that a movie lives up to its hype, even rarer that the hype is transcended by the actual achievement [/INST]  You're absolutely right! It's not uncommon for movies to generate a lot of buzz and excitement before their release, but often the final product doesn't quite live up to the hype. Here are some reasons why this happens:

1. Overhyping: Sometimes, the media and fans can get carried away with the excitement surrounding a movie, building it up to be something it's not. This can create unrealistic expectations that are difficult for the movie to meet.
2. High expectations: When a movie is highly anticipated, the pressure to deliver can be intense. This can lead to a situation where the movie is judged more harshly than it would be if it were a less hyped film.


In [ ]:
# Libera a memória da GPU
from numba import cuda
device = cuda.get_current_device()
device.reset()

In [ ]:
%watermark -a "Data Science Academy"

Author: Data Science Academy



In [ ]:
#%watermark -v -m

In [ ]:
#%watermark --iversions

# Fim